In [15]:
import json
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

annotations = {}


In [16]:
classes_map = {
    "wzmocnienie": 0,
    "osłabienie": 1,
    "odwracanie": 2,
    "Wzmocnienie": 0,
    "Osłabienie": 1,
    "Odwracanie": 2,
    "Odwrócenie": 2,
    "odwrocenie": 2,
    "oslabienie": 1,
    "odwrócenie": 2
}

In [17]:
def loadDataset(path: str, annots: dict[str, list[list]] = None):
    with open(path, 'r', encoding='utf-8') as file:
        if annots == None:
            annots = {}
            for line in file:
                line = json.loads(line)
                annots[line["text"]] = [[[startIndex, endIndex, classes_map[label]] for startIndex, endIndex, label in line["label"]]]
        else:
            for line in file:
                line = json.loads(line)
                annots[line["text"]].append([[startIndex, endIndex, classes_map[label]] for startIndex, endIndex, label in line["label"]])
    return annots

In [18]:
def separateConflicts(annots: dict[str, list[list]]) -> tuple[list[dict], list[dict]]:
    correct = []
    wrong = []

    for text, labels in annots.items():
        if all(label == labels[0] for label in labels):
            correct.append({"text": text, "label": labels[0]})
        else:
            wrong.append({"text": text, "label": "", "old_labels": str(labels)})
    
    return correct, wrong

In [19]:
def classify_words(text, spans):
    words = text.split()
    word_positions = []
    position = 0

    for word in words:
        start_pos = text.find(word, position)
        end_pos = start_pos + len(word)
        word_positions.append((start_pos, end_pos))
        position = end_pos

    word_classes = [3] * len(words)

    for start, end, label in spans:
        for i, (word_start, word_end) in enumerate(word_positions):
            if word_start >= start and word_end <= end:
                word_classes[i] = label

    return word_classes

In [20]:
def kappaCohenResult(annots: dict[str, list[list]]):
    annotators = [[], [], []]
    for text, annotations in annots.items():
        for annotation, annotator in zip(annotations, annotators):
            annotator.extend(classify_words(text, annotation))
    
    print('Kappa Cohen:')
    print(f'Klaudia - Michał: {cohen_kappa_score(annotators[0], annotators[1])}')
    print(f'Klaudia - Kajtek: {cohen_kappa_score(annotators[0], annotators[2])}')
    print(f'Kajtek - Michał: {cohen_kappa_score(annotators[2], annotators[1])}')


In [21]:
def kappaFleissResult(annots: dict[str, list[int | str]]):
    results = []
    for text, annotations in annots.items():
        words = []
        new_sample = [0, 0, 0, 0]
        for annotation in annotations:
            new_sample[annotation] += 1
        results.append(new_sample)
    print(f'Kappa Fleiss: {fleiss_kappa(results)}')

In [22]:
def overlap_length(fragment1, fragment2):
    """Oblicz długość części wspólnej dwóch fragmentów."""
    overlap_start = max(fragment1[0], fragment2[0])
    overlap_end = min(fragment1[1], fragment2[1])
    return max(0, overlap_end - overlap_start)

In [23]:
def overlapRatio(annots: dict[str, list[int | str]]):
    common_score = {0: 0, 1: 0, 2: 0}
    all_score = {0: 0, 1: 0, 2: 0}
    for text, labels in annots.items():
        annotatations = []
        for annotator_labels in labels:
            annotator_annotations = [[False] * len(text) for _ in range(3)]
            for start, end, label in annotator_labels:
                annotator_annotations[label][start:end] = [True] * (end-start)
            annotatations.append(annotator_annotations)

        for i in range(3):
            common = 0
            all = 0
            for j in range(len(text)):
                common_check = True
                all_check = False
                for annotator_annotations in annotatations:
                    if annotator_annotations[i][j]:
                        all_check = True
                    else:
                        common_check = False
                if common_check:
                    common += 1
                if all_check:
                    all += 1
            common_score[i] += common
            all_score[i] += all
    
    print("Overlap Score:")
    print(f"wzmocnienie: {common_score[0] / all_score[0]}")
    print(f"osłabienie: {common_score[1] / all_score[1]}")
    print(f"odwrócenie: {common_score[2] / all_score[2]}")
    print(f"łączne: {sum(common_score.values()) / sum(all_score.values())}")

In [24]:
data = loadDataset('data/michal_fragmenty_1iter.jsonl')
data = loadDataset('data/kajetan_fragmenty_jeden.jsonl', data)

In [25]:
overlapRatio(data)

Overlap Score:
wzmocnienie: 0.2851153039832285
osłabienie: 0.4502923976608187
odwrócenie: 0.837772397094431
łączne: 0.5268614514608859


In [26]:
correct, wrong = separateConflicts(data)

In [27]:
def write_jsonl(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [28]:
write_jsonl("data/first_round_fragments_correct.jsonl", correct)
write_jsonl("data/first_round_fragments_wrong.jsonl", wrong)

# Druga iteracja

In [29]:
data = loadDataset('data/michal_fragmenty_2iter.jsonl')
data = loadDataset('data/kajetan_fragmenty_drugie.jsonl', data)

In [30]:
overlapRatio(data)

Overlap Score:
wzmocnienie: 0.6270928462709284
osłabienie: 0.5333333333333333
odwrócenie: 0.6492753623188405
łączne: 0.6182121971595655


In [31]:
correct, wrong = separateConflicts(data)

In [32]:
write_jsonl("data/second_round_fragments_correct.jsonl", correct)
write_jsonl("data/second_round_fragments_wrong.jsonl", wrong)